# 2章：小売店のデータでデータ加工を行う
Excelなど手入力で作られたデータは基本的に間違いや洩れのある汚いデータ。<br>
ここでは汚いデータを処理する経験を身に着ける。

File:
* uriage.csv
* kokyaku_daicho.xlsx

# ノック11：データを読み込む
CVSデータとExcelデータを取り込む。取り込んだデータを見てみると、入力されたデータの表記ミス、表記方法のばらつき、全角と半角の混在などが目立つ。

In [102]:
import pandas as pd
uriage_data = pd.read_csv("uriage.csv")
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


In [103]:
kokyaku_data = pd.read_excel("kokyaku_daicho.xlsx")
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


# ノック12：データの揺れを見る
売上履歴の```item_name```と```item_price```を抽出してデータの揺れを確認する。<br>
```item_name```では、同じ商品Aでも様々な表記がされているため一つの商品でも複数種類の商品として集計されてしまう。<br>
```item_price```では、データの欠損が見られる。

In [104]:
uriage_data["item_name"].head()

0      商品A
1    商 品 S
2    商 品 a
3      商品Z
4      商品a
Name: item_name, dtype: object

In [105]:
uriage_data["item_price"].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

# ノック13：データに揺れがあるまま集計してみる
ここでは揺れのデータを含んだデータを集計して、データの整合性がいかに重要かを確認する。

In [106]:
uriage_data["purchase_date"] = pd.to_datetime(uriage_data["purchase_date"])
uriage_data["purchase_month"] = uriage_data["purchase_date"].dt.strftime("%Y%m")
res = uriage_data.pivot_table(index="purchase_month", columns="item_name", aggfunc="size", fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


「商品S」や「商品s」のようにデータの揺れがあるため、本来26個の商品が99商品に増えてしまってる。また、```item_name```も同様にデータの揺れがあるので全く意味のない結果が出ることがわかる。

In [107]:
res = uriage_data.pivot_table(index="purchase_month", columns="item_name", values="item_price", aggfunc="sum", fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1400,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,2400,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
201903,0,0,500,1300,1600,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,2300,0,0,0,0,0,0,0,0,1700,...,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,0,1900,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,0,2300,0,0,0,...,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,...,0,0,1500,0,1800,0,0,0,0,0


# ノック14：商品名の揺れを補正する
ここでは、「スペースの有無」、「半角・全角」の揺れを補正する。
```unique```を使って重複データを除外すると、99個のユニークな商品名が存在することが分かる。

In [108]:
print(len(pd.unique(uriage_data.item_name)))

99


それでは早速データをきれいきれい<br>
1行目: 小文字を大文字に変換<br>
2行目: 全角スペースを取り除く<br>
3行目: 半角スペースを取り除く<br>
4行目: ```item_name```順にソート

In [109]:
uriage_data["item_name"] = uriage_data["item_name"].str.upper()
uriage_data["item_name"] = uriage_data["item_name"].str.replace("　", "")
uriage_data["item_name"] = uriage_data["item_name"].str.replace(" ", "")
uriage_data.sort_values(by=["item_name"], ascending=True)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1748,2019-05-19 20:22:22,商品A,100.0,松川綾女,201905
223,2019-06-25 08:13:20,商品A,100.0,板橋隆,201906
1742,2019-06-13 16:03:17,商品A,100.0,小平陽子,201906
1738,2019-02-10 00:28:43,商品A,100.0,松田浩正,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:52,商品Y,NaN,田辺光洋,201904
2881,2019-04-30 14:21:09,商品Y,NaN,高原充則,201904
1525,2019-01-24 10:27:23,商品Y,2500.0,五十嵐春樹,201901
1361,2019-05-28 13:45:32,商品Y,2500.0,大崎ヒカル,201905


揺れを含むデータ補正後は、正しく補正されたかを検証する必要がある。

In [110]:
print(pd.unique(uriage_data["item_name"]))
print(len(pd.unique(uriage_data["item_name"])))

['商品A' '商品S' '商品Z' '商品V' '商品O' '商品U' '商品L' '商品C' '商品I' '商品R' '商品X' '商品G'
 '商品P' '商品Q' '商品Y' '商品N' '商品W' '商品E' '商品K' '商品B' '商品F' '商品D' '商品M' '商品H'
 '商品T' '商品J']
26


# ノック15：金額欠損値の補完をしよう
実際に金額の補完をする場合には、補完方法には、①スタッフに手入力で欠損値を埋めてもらう、②顧客にヒアリングして補完するなどの対応を取る必要がある。<br>
```isnull()```関数で欠損値の有無を確認することが可能。<br>
```axis=0```は列に対して、```axis=1```は行に対しての結果となる。<br>

In [111]:
#uriage_data.isnull().any(axis=0)
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

それでは欠損値の補完を行っていく<br><br>
1行目：```item_price```の欠損値を特定し、Flagを立てる。これにより、```flg_is_null```に欠損値の有無が保持される<br>
2行目：ループ処理を行う。まず、欠損値を含んでいる```item-name```のリストを重複を除いて作成する<br>
　```list()```は変数の値をリスト形式に変換する<br>
　```.loc```関数は条件を付与して、それに合致するデータを抽出できる<br>
　```unique()```で重複データを取り除く。<br>
3行目：欠損していないデータから、欠損を含むリストを対象に正規の金額を取得する。<br>
　```~flg_is_null```は```flg_is_null == False```と同義。つまり、欠損していないデータを指す。<br>
4行目：欠損値を持つ```item_price```に正規の金額を代入する。<br>
5行目：補完処理後の結果を表示

In [112]:
flg_is_null = uriage_data["item_price"].isnull()
for trg in list(uriage_data.loc[flg_is_null, "item_name"].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data["item_name"] == trg), "item_price"].max()
    uriage_data["item_price"].loc[(flg_is_null) & (uriage_data["item_name"]==trg)] = price
uriage_data.head()

C:\Users\tfx746\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904


### 金額の欠損を補完ができたことを検証する

In [113]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

### 補完が正常に行われたことの検証を行う
各商品の金額の最大値と最小値を抽出し、その2つの値が一致することを確認することで補完の正常性を担保する。<br>
```skipna=False```はNaNが存在する場合に、最小値はNaNを結果として表示される。

In [114]:
for trg in list(uriage_data["item_name"].sort_values().unique()):
    print(trg + "\tmaximum price:" + str(uriage_data.loc[uriage_data["item_name"]==trg]["item_price"].max()) + "\tminumum price:" + str(uriage_data.loc[uriage_data["item_name"]==trg]["item_price"].min(skipna=False)))

商品A	maximum price:100.0	minumum price:100.0
商品B	maximum price:200.0	minumum price:200.0
商品C	maximum price:300.0	minumum price:300.0
商品D	maximum price:400.0	minumum price:400.0
商品E	maximum price:500.0	minumum price:500.0
商品F	maximum price:600.0	minumum price:600.0
商品G	maximum price:700.0	minumum price:700.0
商品H	maximum price:800.0	minumum price:800.0
商品I	maximum price:900.0	minumum price:900.0
商品J	maximum price:1000.0	minumum price:1000.0
商品K	maximum price:1100.0	minumum price:1100.0
商品L	maximum price:1200.0	minumum price:1200.0
商品M	maximum price:1300.0	minumum price:1300.0
商品N	maximum price:1400.0	minumum price:1400.0
商品O	maximum price:1500.0	minumum price:1500.0
商品P	maximum price:1600.0	minumum price:1600.0
商品Q	maximum price:1700.0	minumum price:1700.0
商品R	maximum price:1800.0	minumum price:1800.0
商品S	maximum price:1900.0	minumum price:1900.0
商品T	maximum price:2000.0	minumum price:2000.0
商品U	maximum price:2100.0	minumum price:2100.0
商品V	maximum price:2200.0	minumum price:2200.0
商品W	ma

# ノック16："顧客名"の揺れを補正する
顧客台帳と売上履歴の顧客名を比較すると、顧客台帳の顧客名には姓名の間にスペースが含まれており、<br>
売上履歴の顧客名にはスペースが含まれていない。また、スペースには半角と全角が混ざっている。<br>
顧客名以外に2つのデータをつなげるキーが存在しないので、顧客名の補正は必須となる。

In [115]:
kokyaku_data["顧客名"] = kokyaku_data["顧客名"].str.replace("　","")
kokyaku_data["顧客名"] = kokyaku_data["顧客名"].str.replace(" ", "")
kokyaku_data["顧客名"].head()

0    須賀ひとみ
1     岡田敏也
2      芳賀希
3      荻野愛
4     栗田憲一
Name: 顧客名, dtype: object

# ノック17："日付"の揺れを補正する
Excelデータを取り扱う際、書式が異なるデータが存在する場合があることに注意する必要がある。<br>
ここでは、「42782」など数値となっている日付データが含まれている。<br><br>
### 数値データを補正
まず、何件数値データとして取り込まれているかを確認する。

In [116]:
flg_is_serial = kokyaku_data["登録日"].astype("str").str.isdigit()
flg_is_serial.sum()

22

In [117]:
kokyaku_data.loc[flg_is_serial, "登録日"].astype("float")

1      42782.0
3      42872.0
4      43127.0
21     42920.0
27     42901.0
47     42741.0
49     42929.0
53     42833.0
76     43188.0
80     43110.0
99     42885.0
114    43254.0
118    43129.0
122    43206.0
139    42880.0
143    42818.0
155    42754.0
172    43181.0
179    42743.0
183    42940.0
186    43294.0
192    43259.0
Name: 登録日, dtype: float64

In [118]:
fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, "登録日"].astype("float"), unit="D") + pd.to_datetime("1900/01/01")
fromSerial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 登録日, dtype: datetime64[ns]

### 日付として取り込まれてるデータも書式統一

In [119]:
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, "登録日"])
len(fromString)

178

In [120]:
kokyaku_data["登録日"] = pd.concat([fromSerial, fromString])
kokyaku_data

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-19
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-29
...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017-06-20
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018-06-20
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017-04-29
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19


登録日の書式が綺麗に補正されたことが分かる。<br>
この後、登録日から登録月の集計を行ってみる。
## 登録月の集計

In [121]:
kokyaku_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   顧客名      200 non-null    object        
 1   かな       200 non-null    object        
 2   地域       200 non-null    object        
 3   メールアドレス  200 non-null    object        
 4   登録日      200 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 7.9+ KB


### 登録月の集計結果の検証
全データは200件あるため、登録月毎の件数を合計すると200件になる必要がある。

In [122]:
kokyaku_data["登録年月"] = kokyaku_data["登録日"].dt.strftime("%Y%m")
rslt = kokyaku_data.groupby("登録年月").count()["顧客名"]
print(rslt)
print(len(kokyaku_data))

登録年月
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 顧客名, dtype: int64
200


In [123]:
kokyaku_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   顧客名      200 non-null    object        
 1   かな       200 non-null    object        
 2   地域       200 non-null    object        
 3   メールアドレス  200 non-null    object        
 4   登録日      200 non-null    datetime64[ns]
 5   登録年月     200 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 9.5+ KB


### 数値データが残っていないことでも検証可能

In [126]:
flg_is_serial = kokyaku_data["登録日"].astype("str").str.isdigit()
flg_is_serial.sum()

0

# ノック18：顧客名をキーに2つのデータを結合する
第2章では整合性が取れていない2つのデータの結合になるため、共通キーとなるデータを作ってから結合する必要がある。